In [3]:
import pandas as pd
import pandas_profiling
import os
import pickle
import gc
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
import lightgbm as lgb

#データ読み込み
train = pd.read_csv("data_EDA/train.csv")
test = pd.read_csv("data_EDA/test.csv")

In [4]:
X_train = train[['DiabetesPedigreeFunction',
                 'BMI',
                 'Glucose',
                 'Age',
                 'Pregnancies',
                 'BloodPressure_mean', 
                 'SkinThickness_mean',
                 'Insulin_mean',
                ]]
id_train = train[['index']]
y_train = train[['Outcome']]

In [5]:
# 標準化
change_cloumns = ['DiabetesPedigreeFunction',
                 'BMI',
                 'Glucose',
                 'Age',
                 'Pregnancies',
                 'BloodPressure_mean', 
                 'SkinThickness_mean',
                 'Insulin_mean',
                ]

for col in change_cloumns :
    value_min = X_train[col].min()
    value_max = X_train[col].max()
    X_train[col] = (X_train[col] - value_min) / (value_max - value_min)

In [6]:
X_train

,DiabetesPedigreeFunction,BMI,Glucose,Age,Pregnancies,BloodPressure_mean,SkinThickness_mean,Insulin_mean
0,0.138695,0.455371,0.489209,0.521739,0.692308,0.43750,0.473214,0.213894
1,0.042852,0.444655,0.374101,0.021739,0.307692,0.53125,0.473214,0.213894
2,0.061973,0.977454,0.223022,0.108696,0.307692,0.50000,0.761905,0.213894
3,0.026935,0.707838,0.489209,0.521739,0.692308,0.43750,0.473214,0.213894
4,0.064518,0.989802,0.359712,0.521739,0.384615,0.50000,0.880952,0.476950
...,...,...,...,...,...,...,...,...
2995,0.011701,0.571724,0.223022,0.043478,0.307692,0.43750,0.238095,0.213894
2996,0.052923,0.418351,0.625899,0.000000,0.000000,0.65625,0.473214,0.213894
2997,0.264481,0.445837,0.431655,0.021739,0.461538,0.78125,0.690476,0.213894
2998,0.055808,0.554864,0.402878,0.369565,0.153846,0.43750,0.473214,0.213894


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization
from tensorflow.keras.layers import Embedding, Flatten, Concatenate
from tensorflow.keras.callbacks import Earlystopping, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.optimaizers import Adam, SGD

In [9]:
def seed_everything(seed):
    import random
    random.seed(seed)
    os.version['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    session_conf = tf.compat.v1.ConfigProto(
        intra_op_parallelism_threas=1,
        inter_op_parallelism_threas=1
    )
    sess = tf.compat.v1Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
    tf.compat.v1.keras.backend.set_session(sess)

In [11]:
x_tr, x_va, y_tr, y_va = train_test_split(X_train,
                                          y_train,
                                          test_size=0.2,
                                          shuffle=True,
                                          stratify=y_train,
                                          random_state=123)


In [12]:
def create_model():
    input_num = Input(shape=(8,))
    x_num = Dense(10, activation='relu')(input_num)
    x_num = BatchNormalization()(x_num)
    x_num = Dropout(0.3)(x_num)
    x_num = Dense(10, activation='relu')(x_num)
    x_num = BatchNormalization()(x_num)
    x_num = Dropout(0.2)(x_num)
    x_num = Dense(5, activation='relu')(x_num)
    x_num = BatchNormalization()(x_num)
    x_num = Dropout(0.1)(x_num)
    out = Dense(1, activation='sigmoid')(x_num)
    
    model = Model(input=input_num, output=out)
    
    model.compile(
        optimizer='Adam',
        loss='binarl_crossentropy',
        metrics=['binary_crossentropy'],
    )
    
    return model

In [ ]:
model = create_model()
model.summary()

In [ ]:
seed_evrything(seed=123)
model = create_model()
model.fit(x=x_tr,
          y=y_tr,
         validation_data=(x_va, y_va),
         batch_size=8,
         epochs=10000,
         callbacks=[ModelCheckpoint(filepath='model')])